In [40]:
import cv2
import numpy as np
import os
import scipy.io as sio
from cvprlab_python.lab3.extract_random import extract_random


In [42]:
def extract_color_descriptor(img):
    # Compute the average red, green, and blue values as a basic color descriptor
    R = np.mean(img[:, :, 2])  # Note: OpenCV uses BGR format
    G = np.mean(img[:, :, 1])
    B = np.mean(img[:, :, 0])
    return np.array([R, G, B])

In [43]:
DATASET_FOLDER = "cvprlab_python/MSRC_ObjCategImageDatabase_v2"
OUT_FOLDER = "cvprlab_python/lab3/descriptors"
OUT_SUBFOLDER = 'globalRGBhisto'

if not os.path.exists(OUT_FOLDER):
    os.makedirs(OUT_FOLDER)

# Extract Image Descriptors from the Dataset
os.makedirs(os.path.join(OUT_FOLDER, OUT_SUBFOLDER), exist_ok=True)

In [45]:
for filename in os.listdir(os.path.join(DATASET_FOLDER, 'Images')):
    if filename.endswith(".bmp"):
        print(f"Processing file {filename}")
        img_path = os.path.join(DATASET_FOLDER, 'Images', filename)
        img = cv2.imread(img_path).astype(np.float64) / 255.0  # Normalize the image
        fout = os.path.join(OUT_FOLDER, OUT_SUBFOLDER, filename.replace('.bmp', '.npy'))
        
        # Call extractRandom (or another feature extraction function) to get the descriptor
        F = extract_random(img)        

        # Save the descriptor to a .npy file
        np.save(fout, F)


Processing file 17_15_s.bmp


FileNotFoundError: [Errno 2] No such file or directory: 'cvprlab_python/lab3/descriptors/globalRGBhisto/17_15_s.npy'